Networks - Final Project

"Is there an optimal road to close and convert to a public use space in Barcelona that minimizes the impact on traffic?"

-Clarice Mottet, Amber Walker, Mox Ballo

Outline:
-import Barcelona road network data into programming environment
-subset the data to be only a neighborhood of Barcelona
-determine alpha and beta parameters for latency function
-create a dataset that is a very simple road network
-write a function that calculates latency between nodes



https://nbviewer.org/github/PyTrans/Urban-Network-Analysis/blob/master/Trip_Assignment-Frank-Wolfe_Algorithm.ipynb

SUPER IMPORTANT

I saw in the TransportationNetworks.py file that their initial alpha was .5 but everywhere said it should be .15, so I changed it to .15 in my file.

In [4]:
#libraries

import pandas as pd
import numpy as np
import networkx as nx
import networkx as nx
import scipy.integrate as integrate 
from scipy.optimize import minimize_scalar
import matplotlib.pyplot as plt
from scipy.misc import derivative
import TransportationNetworks as tn

path_in_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/inputs'
path_out_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/outputs'
path_out_results_ = r'/home/clarice/Documents/VSCode/Term2_Networks/final_project/networks_finalproject/outputs/results'

In [ ]:
#functions

#from website to calculate latency
def BPR(t0, xa, ca, alpha, beta):
    ta = t0*(1+alpha*(xa/ca)**beta)
    return ta

#from website: method to calculate nash equilibrium (when SO = False)
def calculateZ(theta, network, SO):
    z = 0
    for linkKey, linkVal in network.items():
        t0 = linkVal['t0']
        ca = linkVal['capa']
        beta = linkVal['beta']
        alpha = linkVal['alpha']
        aux = linkVal['auxiliary'][-1]
        flow = linkVal['flow'][-1]
        
        if SO == False:
            z += integrate.quad(lambda x: BPR(t0, x, ca, alpha, beta), 0, flow+theta*(aux-flow))[0]
        elif SO == True:
            z += list(map(lambda x : x * BPR(t0, x, ca, alpha, beta), [flow+theta*(aux-flow)]))[0]
    return z

#from website: finds nash equilibrium
def lineSearch(network, SO):
    theta = minimize_scalar(lambda x: calculateZ(x, network, SO), bounds = (0,1), method = 'Bounded')
    return theta.x

In [ ]:
#functions - user, based on jupyter notebook

def initialize(link_file, trip_file, node_file):
    #set objective and open the network
    SO = False # True - System optimum, False - User equilibrium
    barcelonaSubset = tn.Network(link_file, trip_file, node_file, SO)

    #start of initialization

    # define output variables, network and fwResult
    network = {(u,v): {'t0':d['object'].t0, 'alpha':d['object'].alpha, \
            'beta':d['object'].beta, 'capa':d['object'].capacity, 'flow':[], \
            'auxiliary':[], 'cost':[]} for (u, v, d) in barcelonaSubset.graph.edges(data=True)}

    fwResult = {'theta':[], 'z':[]}

    # initial all-or-nothing assignment and update link travel time(cost)
    barcelonaSubset.all_or_nothing_assignment()
    barcelonaSubset.update_linkcost()

    for linkKey, linkVal in network.items():
        linkVal['cost'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['weight'])
        linkVal['auxiliary'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)
        linkVal['flow'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)

    return barcelonaSubset, network, fwResult

#calculate nash equilibrium
def calc_nash(barcelonaSubset, network, fwResult):
    SO=False
    ## iterations
    iterNum=0
    iteration = True
    while iteration:
        iterNum += 1
        barcelonaSubset.all_or_nothing_assignment()
        barcelonaSubset.update_linkcost()
        
        # set auxiliary flow using updated link flow
        for linkKey, linkVal in network.items():
            linkVal['auxiliary'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol)
            
        # getting optimal move size (theta)
        theta = lineSearch(network, SO)
        fwResult['theta'].append(theta)
        
        # set link flow (move) based on the theta, auxiliary flow, and link flow of previous iteration
        for linkKey, linkVal in network.items():
            aux = linkVal['auxiliary'][-1]
            flow = linkVal['flow'][-1]
            linkVal['flow'].append(flow + theta*(aux-flow))
            
            barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].vol =  flow + theta * (aux - flow)
            barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].flow = flow + theta * (aux - flow)
            
        # update link travel time
        barcelonaSubset.update_linkcost()
        
        # calculate objective function value
        z=0
        for linkKey, linkVal in network.items():
            linkVal['cost'].append(barcelonaSubset.graph[linkKey[0]][linkKey[1]]['weight'])
            totalcost = barcelonaSubset.graph[linkKey[0]][linkKey[1]]['object'].get_objective_function()
            z+=totalcost
            
        fwResult['z'].append(z)        
            
        # convergence test
        if iterNum == 1:
            iteration = True
        else:
            if abs(fwResult['z'][-2] - fwResult['z'][-1]) <= 0.001 or iterNum==3000:
                iteration = False

    return barcelonaSubset, network, fwResult

In [ ]:
#functions - user

#remove edges from consideration

#create mini block list

#create super block list

#calc total latency

#store and export results



In [ ]:
#import data into programming environment
directory = ".\\Data\\TransportationNetworks\\barcelonaSubset\\"
link_file = '{}Barcelona_net.tntp'.format(directory)
trip_file = '{}Barcelona_trips.tntp'.format(directory)
node_file = '{}barcelonaSubset_node.tntp'.format(directory)

#need to create a node file

#subset to just one neighborhood of Barcelona


In [ ]:
#

# initialization

In [ ]:


#initialize graph
barcelonaSubset, network, fwResult = initialize(link_file, trip_file, node_file)

#find nash equilibrium
barcelonaSubset, network, fwResult = calc_nash(barcelonaSubset, network, fwResult)

#calculate latency

#export flow
#export latency